# Generate QnA synthetic dataset from CSV


In [ ]:
from dotenv import load_dotenv
import os, shutil, random
from langchain_community.document_loaders.csv_loader import CSVLoader
from util.preprocess import convert_html_to_md
from util.common_utils import get_language_code

load_dotenv()

raw_data_dir = "../raw_data"
file_path = f"{raw_data_dir}/csv/ko-cs-center-info.csv"

DOMAIN = "CS (Customer Support)"
LANGUAGE = "English" # You can change your language here. e.g., "Korean", "Japanese", "Chinese"
LANGUAGE_CODE = get_language_code(LANGUAGE)
print(f"Domain: {DOMAIN}, Language: {LANGUAGE}, Language Code: {LANGUAGE_CODE}")


## 1. Read & Preprocess CSV file
---

In [ ]:
%%time

loader = CSVLoader(
    file_path=file_path,
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
    },    
)
docs = loader.load()

In [ ]:
preprocessed_docs = []
for doc in docs:
    md = convert_html_to_md(doc.page_content)
    preprocessed_docs.append(md)

print(preprocessed_docs[0])

## 2. Construct QnA Pairs
---

In [ ]:
aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from util.qa_pair import get_qna_prompt_template, QAPair

llm = AzureChatOpenAI(
    temperature=0, 
    max_tokens=1024,
    openai_api_version=aoai_api_version,
    azure_deployment=aoai_deployment_name                    
)

parser = JsonOutputParser(pydantic_object=QAPair)
prompt = get_qna_prompt_template(LANGUAGE)

chain = prompt | llm | parser

Prepare input batch

In [ ]:
input_batch = []

for doc in preprocessed_docs:
    dic = {"context": doc, "domain": DOMAIN, "num_questions": "1"}
    input_batch.append(dic)

In [ ]:
%%time
qa_pair = chain.batch(input_batch, {"max_concurrency": 5})

In [ ]:
# Remove duplicate items
qa_pair = [dict(t) for t in {tuple(d.items()) for d in qa_pair}]


## 3. Save to jsonl for fine-tuning
---

In [ ]:
import json
from util.common_utils import convert_to_oai_format, save_jsonl

output_dir = './dataset'
os.makedirs(output_dir, exist_ok=True)

system_prompt_msg = f"""You are the SME (Subject Matter Expert) in {DOMAIN}. Please answer the questions accurately. If the question is in {LANGUAGE}, write your answer in {LANGUAGE}."""

save_filename = "cs-center-info"
oai_qa_pair = convert_to_oai_format(qa_pair, system_prompt_msg=system_prompt_msg)

#save_jsonl(qa_pair, f"{output_dir}/{save_filename}.jsonl")
save_jsonl(oai_qa_pair, f"{output_dir}/{save_filename}-oai.jsonl")